## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as func
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
#from spark_sklearn import GridSearchCV
import pandas
import numpy as np

In [2]:
spark = SparkSession.builder.master('spark://192.168.1.85:7077').appName('House_analysis').getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/16 16:38:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Data load

In [3]:
df = spark.read.csv('./data/house_data', header=True, inferSchema=True)
df.toPandas()

23/05/16 16:38:58 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 1) (172.29.174.150 executor 0): java.io.FileNotFoundException: 
File file:/mnt/Arquivos Trabalho/Projetos/Python/competição-kaggle1/data/house_data/part-00000-9046f244-45ba-444e-8ed5-d8bdfd262ec5-c000.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:648)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.c

23/05/16 16:38:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


23/05/16 16:38:59 WARN TaskSetManager: Lost task 0.0 in stage 2.0 (TID 4) (172.29.174.150 executor 0): java.io.FileNotFoundException: 
File file:/mnt/Arquivos Trabalho/Projetos/Python/competição-kaggle1/data/house_data/part-00000-9046f244-45ba-444e-8ed5-d8bdfd262ec5-c000.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:648)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.c

Py4JJavaError: An error occurred while calling o34.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 8) (172.29.174.150 executor 0): java.io.FileNotFoundException: 
File file:/mnt/Arquivos Trabalho/Projetos/Python/competição-kaggle1/data/house_data/part-00000-9046f244-45ba-444e-8ed5-d8bdfd262ec5-c000.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:648)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRD

In [8]:
df_eval = df[df["SalePrice"].isNotNull()]
final_test_df = df[df["SalePrice"].isNull()]
df_eval.toPandas()

23/05/16 16:37:29 WARN TaskSetManager: Lost task 1.0 in stage 7.0 (TID 22) (172.29.174.150 executor 0): java.io.FileNotFoundException: 
File file:/mnt/Arquivos Trabalho/Projetos/Python/competição-kaggle1/data/house_data/part-00001-9046f244-45ba-444e-8ed5-d8bdfd262ec5-c000.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:648)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.

,DwellingType,ZoneType,LotFrontage,LotArea,Street,Alley,LotShape,Flatness,Utilities,LotConfig,...,NonbuiltArea,BuiltAreaRatio,IsDuplex,SoldAfterRemodel,BathroomsBedroomsRatio,PercentFrontage,ImpactView,RoomsMeanSize,RegularRooms,GarageAreaPerCars
0,2-STORY 1946 & NEWER,Residential Low Density,65,8450,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,6740,0.202367,1,0,1.000000,0.007692,0.038462,213.750000,4,274.000000
1,1-STORY 1946 & NEWER ALL STYLES,Residential Low Density,80,9600,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,FR2,...,8338,0.131458,0,0,1.500000,0.008333,0.066667,210.333333,2,230.000000
2,2-STORY 1946 & NEWER,Residential Low Density,68,11250,Paved,Not Alley Access,Slightly irregular,Near Flat/Level,All public Utilities,Inside,...,9464,0.158756,1,1,1.000000,0.006044,0.030222,297.666667,2,304.000000
3,2-STORY 1945 & OLDER,Residential Low Density,60,9550,Paved,Not Alley Access,Slightly irregular,Near Flat/Level,All public Utilities,Corner,...,7833,0.179791,1,1,3.000000,0.006283,0.031414,245.285714,3,214.000000
4,2-STORY 1946 & NEWER,Residential Low Density,84,14260,Paved,Not Alley Access,Slightly irregular,Near Flat/Level,All public Utilities,FR2,...,12062,0.154137,1,0,1.333333,0.005891,0.029453,244.222222,4,278.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2-STORY 1946 & NEWER,Residential Low Density,62,7917,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,6270,0.208033,1,1,1.000000,0.007831,0.039156,235.285714,3,230.000000
1456,1-STORY 1946 & NEWER ALL STYLES,Residential Low Density,85,13175,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,11102,0.157343,0,1,1.500000,0.006452,0.038710,296.142857,3,250.000000
1457,2-STORY 1945 & OLDER,Residential Low Density,66,9042,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,6702,0.258792,1,1,2.000000,0.007299,0.065693,260.000000,4,252.000000
1458,1-STORY 1946 & NEWER ALL STYLES,Residential Low Density,68,9717,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,8639,0.110940,0,1,2.000000,0.006998,0.041988,215.600000,2,240.000000


In [9]:
final_test_df.toPandas()

23/05/16 16:37:38 WARN TaskSetManager: Lost task 1.0 in stage 8.0 (TID 26) (172.29.174.150 executor 0): java.io.FileNotFoundException: 
File file:/mnt/Arquivos Trabalho/Projetos/Python/competição-kaggle1/data/house_data/part-00001-9046f244-45ba-444e-8ed5-d8bdfd262ec5-c000.csv does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:648)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:212)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:270)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.

,DwellingType,ZoneType,LotFrontage,LotArea,Street,Alley,LotShape,Flatness,Utilities,LotConfig,...,NonbuiltArea,BuiltAreaRatio,IsDuplex,SoldAfterRemodel,BathroomsBedroomsRatio,PercentFrontage,ImpactView,RoomsMeanSize,RegularRooms,GarageAreaPerCars
0,1-STORY 1946 & NEWER ALL STYLES,Residential High Density,80,11622,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,10726,0.077095,0,0,2.0,0.006883,0.041301,179.200000,2,730.000000
1,1-STORY 1946 & NEWER ALL STYLES,Residential Low Density,81,14267,Paved,Not Alley Access,Slightly irregular,Near Flat/Level,All public Utilities,Corner,...,12938,0.093152,0,0,1.5,0.005677,0.034065,221.500000,2,312.000000
2,2-STORY 1946 & NEWER,Residential Low Density,74,13830,Paved,Not Alley Access,Slightly irregular,Near Flat/Level,All public Utilities,Inside,...,12201,0.117787,1,1,1.0,0.005351,0.026753,271.500000,2,241.000000
3,2-STORY 1946 & NEWER,Residential Low Density,78,9978,Paved,Not Alley Access,Slightly irregular,Near Flat/Level,All public Utilities,Inside,...,8374,0.160754,1,0,1.0,0.007817,0.046903,229.142857,3,235.000000
4,1-STORY PUD (Planned Unit Development) - 1946 ...,Residential Low Density,43,5005,Paved,Not Alley Access,Slightly irregular,Hillside,All public Utilities,Inside,...,3725,0.255744,0,0,1.0,0.008591,0.042957,256.000000,2,253.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2-STORY PUD - 1946 & NEWER,Residential Medium Density,21,1936,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,844,0.564050,1,0,1.5,0.010847,0.075930,218.400000,1,0.000000
1455,2-STORY PUD - 1946 & NEWER,Residential Medium Density,21,1894,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,802,0.576558,1,0,1.5,0.011088,0.055438,182.000000,2,286.000000
1456,1-STORY 1946 & NEWER ALL STYLES,Residential Low Density,160,20000,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,18776,0.061200,0,1,4.0,0.008000,0.056000,174.857143,2,288.000000
1457,SPLIT FOYER,Residential Low Density,62,10441,Paved,Not Alley Access,Regular,Near Flat/Level,All public Utilities,Inside,...,9471,0.092903,0,0,3.0,0.005938,0.029691,161.666667,2,0.000000


## Modeling

In [6]:
numeric_cols = [col for col, dtype in df.dtypes if dtype != 'string']
numeric_cols.remove('SalePrice')
string_cols = [col for col, dtype in df.dtypes if dtype == 'string']

In [24]:
col_indexer_names = [x+"Indexer" for x in string_cols]
col_onehot_names = [x+"OneHot" for x in string_cols]

indexer = StringIndexer(inputCols=string_cols, outputCols=col_indexer_names)
encoder = OneHotEncoder(inputCols=col_indexer_names, outputCols=col_onehot_names)
assembler = VectorAssembler(inputCols=numeric_cols+col_onehot_names, outputCol='features')
pipeline = Pipeline(stages=[indexer, encoder, assembler])

In [25]:
pipe_model = pipeline.fit(df)
df_eval = pipe_model.transform(df_eval)

### Random Forest

In [28]:
rf = RandomForestRegressor(labelCol="SalePrice", featuresCol="features", )

grid = ParamGridBuilder()
grid = grid.addGrid(rf.numTrees, np.arange(100, 501, 100))
grid = grid.addGrid(rf.featureSubsetStrategy, ['sqrt', 'log2', 1, 0.75, 0.5, 0.25])
grid = grid.addGrid(rf.bootstrap, [True, False])
grid = grid.build()

In [47]:
cv_model = CrossValidator(estimator=rf, estimatorParamMaps=grid, evaluator=RegressionEvaluator(labelCol="SalePrice", metricName='rmse'), numFolds=10, parallelism=5)
cv_model = cv.fit(dataset=df_eval)

23/04/28 20:28:46 WARN DAGScheduler: Broadcasting large task binary with size 1268.1 KiB
23/04/28 20:28:47 WARN DAGScheduler: Broadcasting large task binary with size 1290.9 KiB
23/04/28 20:28:49 WARN DAGScheduler: Broadcasting large task binary with size 1293.8 KiB
23/04/28 20:28:49 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
23/04/28 20:28:51 WARN DAGScheduler: Broadcasting large task binary with size 1296.6 KiB
23/04/28 20:28:51 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
23/04/28 20:28:52 WARN DAGScheduler: Broadcasting large task binary with size 1035.2 KiB
23/04/28 20:28:53 WARN DAGScheduler: Broadcasting large task binary with size 1757.7 KiB
23/04/28 20:28:54 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
23/04/28 20:28:55 WARN DAGScheduler: Broadcasting large task binary with size 1034.6 KiB
23/04/28 20:28:56 WARN DAGScheduler: Broadcasting large task binary with size 1769.8 KiB
23/04/28 20:28:56 WARN DAGSche

In [50]:
cv_model.avgMetrics

[32096.111524660286,
 32279.2927153402,
 31844.228776590317,
 32147.486488311817,
 32015.187533787903,
 31997.422024942625]

In [55]:
cv_model.bestModel

RandomForestRegressionModel: uid=RandomForestRegressor_4345b270e34e, numTrees=200, numFeatures=254

In [63]:
len(cv_model.extractParamMap().items())

6

In [105]:
train, test = df_eval.randomSplit([0.9, 0.1])
train.count(), test.count()

(1308, 152)

In [106]:

rf_model = rf.fit(train)

23/04/07 22:17:51 WARN DAGScheduler: Broadcasting large task binary with size 1239.4 KiB


In [107]:
results = rf_model.transform(test)

In [108]:
print("R2:", r2_score(np.array(results[['SalePrice']].collect()), np.array(results[['prediction']].collect())))
print("MAE:", mean_absolute_error(np.array(results[['SalePrice']].collect()), np.array(results[['prediction']].collect())))
print("RMSE:", mean_squared_error(np.array(results[['SalePrice']].collect()), np.array(results[['prediction']].collect())) ** (1/2))

R2: 0.8811069103377805
MAE: 16008.067032715726
RMSE: 22100.15958919812


In [109]:
final_test_df = pipe_model.transform(final_test_df)

In [110]:
rf = RandomForestRegressor(numTrees=100, labelCol="SalePrice", featuresCol="features")
rf_model = rf.fit(df_eval)

23/04/07 22:17:54 WARN DAGScheduler: Broadcasting large task binary with size 1344.1 KiB


In [111]:
results_to_submit = rf_model.transform(final_test_df)

In [112]:
results_to_submit = results_to_submit[['prediction']].withColumnRenamed('prediction', 'SalePrice')
results_to_submit.show()

+------------------+
|         SalePrice|
+------------------+
|126053.91164766779|
| 145333.5734536947|
| 173196.3568250724|
|181667.25960022534|
|210986.88413332804|
|180133.26180332105|
|161756.41270636453|
|168963.01883397638|
|176788.15962884467|
| 124727.4086450906|
|198265.53304276185|
|112912.30855791138|
|114647.11298737467|
|160103.92951713438|
| 141137.2874513425|
|377475.16232445254|
|257701.73528888784|
| 312498.1694346844|
|289957.67746455566|
|424606.52677407884|
+------------------+
only showing top 20 rows



In [113]:
results_to_submit = results_to_submit.toPandas()
results_to_submit["Id"] = range(1461, 1461+len(results_to_submit))
results_to_submit = results_to_submit[["Id", "SalePrice"]]
results_to_submit

,Id,SalePrice
0,1461,126053.911648
1,1462,145333.573454
2,1463,173196.356825
3,1464,181667.259600
4,1465,210986.884133
...,...,...
1454,2915,100692.886521
1455,2916,107950.505939
1456,2917,151569.452208
1457,2918,120829.640975


In [114]:
results_to_submit.to_csv('data/submition_v3.csv', index=False)